In [3]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# 引入你的 src 模組
from src.utils.data_loader import prepare_data
from src.engine.trainer import train_v11
from src.engine.evaluator import evaluate_model

# 設定繪圖風格
sns.set_style("whitegrid")
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'Microsoft JhengHei']  # 解決中文亂碼
plt.rcParams['axes.unicode_minus'] = False  #%%


In [4]:
# 硬體設定
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {DEVICE}")

# 資料與模型參數
DATASET_PATH = Path("./dataset/USD_TWD.csv")
HORIZON = 3  # 預測未來 3 天
LOOKBACK = 30  # 回看過去 30 天
NUM_EPOCHS = 120  # 訓練輪數
LR = 0.001  # 學習率
SEED = 42  # 固定種子

# 設定隨機性 (Reproducibility)
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

Using Device: cuda


In [7]:
if not DATASET_PATH.exists():
    print(f"[Error]: Dataset not found at {DATASET_PATH}")
else:
    print("[Loading Data]")
    df = pd.read_csv(DATASET_PATH)

    # 呼叫 src 裡的函式
    train_loader, test_loader, scalers_raw, _, _, _, _ = prepare_data(
        df, lookback=LOOKBACK, horizon=HORIZON
    )

    # 檢查一下 Batch
    sample = next(iter(train_loader))
    print(f"[Loading Data] Data Loaded! Train Batches: {len(train_loader)}, Test Batches: {len(test_loader)}")
    print(f"[Loading Data] Input Shape: {sample['raw_input'].shape}, Target Shape: {sample['target'].shape}")

[Loading Data]
[Loading Data] Data Loaded! Train Batches: 168, Test Batches: 42
[Loading Data] Input Shape: torch.Size([32, 30, 2]), Target Shape: torch.Size([32, 3])


In [8]:
print(f"[Training] Starting Training for {NUM_EPOCHS} epochs...")

model = train_v11(
    train_loader=train_loader,
    test_loader=test_loader,
    device=DEVICE,
    horizon=HORIZON,
    num_epochs=NUM_EPOCHS,
    lr=LR
)

print("[Training] Training Completed.")

[Training] Starting Training for 120 epochs...

[Training] Enhanced DLinear...
  Epoch 20 | Loss: 0.3196 | Trend W: -0.032 | Seas W: 0.067
  Epoch 40 | Loss: 0.2931 | Trend W: -0.052 | Seas W: 0.083
  Epoch 60 | Loss: 0.2735 | Trend W: -0.066 | Seas W: 0.103
  Epoch 80 | Loss: 0.2647 | Trend W: -0.074 | Seas W: 0.114
  Epoch 100 | Loss: 0.2590 | Trend W: -0.082 | Seas W: 0.121
  Epoch 120 | Loss: 0.2569 | Trend W: -0.083 | Seas W: 0.122
[Training] Training Completed.


In [9]:
print("[Evaluation] Running Evaluation...")

evaluate_model(
    model=model,
    test_loader=test_loader,
    scaler=scalers_raw['target'],  # 記得只傳 target scaler
    device=DEVICE,
    horizon=HORIZON
)

[Evaluation] Running Evaluation...

 FINAL SEQUENCE FORECAST (Horizon=3): Detailed Breakdown
   (Weights -> Trend: -0.0830 | Seasonal: 0.1222)
Metric               | Pure DLinear    | Enhanced        | Improvement    
-----------------------------------------------------------------------------------------------
R2 Score (Overall)   | 0.6960          | 0.7924          | +0.0964 ✅
RMSE (Overall)       | 1.9632          | 1.6224          | -0.3408 ✅
-----------------------------------------------------------------------------------------------
Direction Accuracy (Win Rate)
  Average (All Days) | 0.5944          | 0.7219          | +0.1275 ✅
  Day 1 (T+1)        | 0.5422          | 0.7072          | +0.1650 ✅
  Day 2 (T+2)        | 0.6122          | 0.7300          | +0.1179 ✅
  Day 3 (T+3)        | 0.6289          | 0.7285          | +0.0996 ✅
-----------------------------------------------------------------------------------------------
High Volatility Acc  | 0.6984          | 0.8847   